## 基于GP的集成学习

GP（遗传编程）是一种基于种群的优化算法，非常适合用于集成学习。


### 评估函数
对于集成学习，多样性非常重要。因此，在评估个体时，建议使用随机决策树或其他随机性较高的算法。这样可以产生多样性的GP个体。

In [6]:
from sklearn.datasets import load_iris

# 假设我们有一个分类问题，输入为X，输出为y
X,y=load_iris(return_X_y=True)

# 求解问题的目标是最大化分类精度
def evalFeatureEngineering(trees):
    # 创建新特征
    new_features = []
    for tree_id, tree in enumerate(trees):
        func = gp.compile(expr=tree, pset=pset)
        new_features.append([func(*record) for record in X])
    
    # 转置新特征数组
    new_features = np.transpose(np.array(new_features))
    
    # 使用决策树分类器
    clf = DecisionTreeClassifier(splitter="random")
    clf.fit(new_features, y)
    
    # 使用交叉验证计算误差
    scores = cross_val_score(clf, new_features, y, cv=5)
    
    # 返回平均分类精度
    return scores.mean(),

### 多树GP
集成学习中可以选择使用单树GP或多树GP，两者都是可行的。在本教程中，我们使用多树GP来表示个体。

In [7]:
import operator
import random
import numpy as np
from deap import base, creator, tools, gp, algorithms
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

# 创建GP框架的基本组件
pset = gp.PrimitiveSet("MAIN", X.shape[1])
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(operator.neg, 1)
pset.addEphemeralConstant("rand101", lambda: random.random() * 2 - 1)

# 创建一个适应度类和个体类，个体由多棵树组成
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

C:\Users\zhenl\anaconda3\Lib\site-packages\deap\gp.py:254: RuntimeWarning: Ephemeral rand101 function cannot be pickled because its generating function is a lambda function. Use functools.partial instead.
  warnings.warn("Ephemeral {name} function cannot be "
C:\Users\zhenl\anaconda3\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\zhenl\anaconda3\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


### 交叉/变异算子
交叉和变异算子无需额外修改。

In [8]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)

# 初始化每个个体为一个包含多棵树的列表
def initIndividual(container, func, size):
    return container(gp.PrimitiveTree(func()) for _ in range(size))

# 交叉和变异算子需要能够处理个体的列表结构
def cxOnePointListOfTrees(ind1, ind2):
    for tree1, tree2 in zip(ind1, ind2):
        gp.cxOnePoint(tree1, tree2)
    return ind1, ind2

def mutUniformListOfTrees(individual, expr, pset):
    for tree in individual:
        gp.mutUniform(tree, expr=expr, pset=pset)
    return individual,

toolbox.register("individual", initIndividual, creator.Individual, toolbox.expr, size=3)  # 假设我们创建3个特征
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evalFeatureEngineering)  # 假定这个函数已经定义
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", cxOnePointListOfTrees)
toolbox.register("mutate", mutUniformListOfTrees, expr=toolbox.expr, pset=pset)
toolbox.register("compile", gp.compile, pset=pset)

### 存档机制
对于集成学习，我们需要保存最佳的个体集合，而不仅仅是最佳个体。因此，我们需要将HallOfFame的存档数量从1提高到20。

In [9]:
# 运行遗传编程算法
population = toolbox.population(n=50)
hof = tools.HallOfFame(20)

# 统计和日志
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

pop, log = algorithms.eaSimple(population, toolbox, 0.5, 0.2, 40, stats=stats, halloffame=hof, verbose=True)

gen	nevals	avg     	std      	min	max     
0  	50    	0.905333	0.0798777	0.5	0.966667
1  	27    	0.933733	0.0426544	0.693333	0.98    
2  	31    	0.9304  	0.0702903	0.506667	0.966667
3  	32    	0.941467	0.0205822	0.886667	0.966667
4  	32    	0.941333	0.0468757	0.653333	0.973333
5  	33    	0.945733	0.0244044	0.84    	0.98    
6  	27    	0.9488  	0.0263461	0.82    	0.98    
7  	28    	0.946667	0.0338887	0.82    	0.98    
8  	33    	0.9528  	0.0221446	0.86    	0.973333
9  	23    	0.954667	0.0305214	0.773333	0.973333
10 	33    	0.9236  	0.10487  	0.34    	0.98    
11 	26    	0.954267	0.0275845	0.873333	0.98    
12 	32    	0.9364  	0.0873024	0.426667	0.98    
13 	30    	0.952   	0.0357274	0.766667	0.986667
14 	35    	0.947733	0.0453992	0.693333	0.986667
15 	27    	0.956533	0.0428172	0.686667	0.986667
16 	30    	0.9524  	0.0371665	0.773333	0.986667
17 	24    	0.956667	0.0246667	0.893333	0.986667
18 	31    	0.9548  	0.0254485	0.853333	0.98    
19 	33    	0.954667	0.0331729	0.76    	0.98    
20

### 集成预测
最后，根据最佳个体集合，可以使用投票法进行集成预测。

In [10]:
# 使用保存的最佳个体集合进行集成预测
def ensemblePredict(hof, X):
    # 对每个个体，使用它创建的特征进行预测
    predictions = []
    for ind in hof:
        print('Individual:', [str(tree) for tree in ind])
        # 创建新特征
        new_features = []
        for tree_id, tree in enumerate(ind):
            func = gp.compile(expr=tree, pset=pset)
            new_features.append([func(*record) for record in X])
        
        # 转置新特征数组
        new_features = np.transpose(np.array(new_features))
        
        # 在这里我们只是简单的使用了新特征，并没有使用原始特征
        # 在实际应用中可能需要结合新老特征
        clf = DecisionTreeClassifier(splitter='random')
        clf.fit(new_features, y)
        predictions.append(clf.predict(new_features))

    # 对预测结果进行集成
    # 这里我们使用简单多数投票，也可以根据需要使用其他集成方法
    ensemble_pred = np.apply_along_axis(lambda x: np.bincount(x, minlength=3).argmax(), axis=0, arr=np.array(predictions))
    return ensemble_pred

# 对集成预测函数的测试
ensemble_pred = ensemblePredict(hof, X)
print('Ensemble prediction:', ensemble_pred[:5])

Individual: ['add(sub(mul(ARG3, ARG2), ARG0), mul(ARG3, ARG2))', 'sub(ARG0, add(ARG2, ARG0))', 'neg(-0.9478611294126922)']
Individual: ['add(sub(mul(ARG3, ARG2), ARG0), ARG2)', 'sub(ARG0, ARG0)', 'neg(-0.9478611294126922)']
Individual: ['add(sub(ARG2, ARG0), mul(mul(ARG3, ARG2), ARG2))', 'sub(ARG0, ARG0)', 'neg(-0.9478611294126922)']
Individual: ['add(sub(mul(ARG3, ARG2), ARG0), mul(ARG3, ARG2))', 'sub(add(ARG0, 0.6621306191031775), ARG0)', 'neg(-0.9478611294126922)']
Individual: ['add(sub(mul(ARG3, ARG2), ARG0), mul(ARG3, ARG2))', 'sub(ARG0, add(ARG2, 0.6621306191031775))', 'neg(ARG0)']
Individual: ['add(sub(mul(ARG3, ARG2), ARG0), mul(ARG3, ARG2))', 'sub(ARG0, ARG0)', 'mul(-0.9478611294126922, neg(ARG2))']
Individual: ['add(sub(mul(ARG3, neg(sub(-0.13698934661694762, 0.5264481929444698))), ARG3), mul(ARG3, ARG2))', 'sub(mul(ARG2, ARG2), add(ARG1, ARG1))', 'sub(ARG1, ARG1)']
Individual: ['add(sub(sub(mul(ARG3, ARG2), ARG0), ARG3), ARG2)', 'sub(ARG0, ARG0)', 'neg(ARG0)']
Individual: ['